In [41]:
import spacy
import requests
import datetime
import pyowm
import random
from mechanize import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from math import floor
import nltk

In [169]:
def getWeather(location, date):
    """
    [str, datetime] -> dict
    Finish time for more than 5 days
    """
    openWeatherKey = "441128368845c5854135b6a09d13ca66"
    owm = pyowm.OWM(openWeatherKey) 
    
    weather_dict = dict()
    diff = abs(datetime.datetime.now().day - date.day)
    if  diff < 5: #if less than 5 days use open weather api
        fc = owm.three_hours_forecast(location)
        f = fc.get_weather_at(date)
        weather_dict.update({"temp_celsius_prec":f.get_temperature(unit="celsius")["temp"]})
        weather_dict.update({"temp_fahr_prec":f.get_temperature(unit="fahrenheit")["temp"]})
        weather_dict.update({"will_rain": fc.will_be_rainy_at(date)})
        weather_dict.update({"will_snow": fc.will_be_snowy_at(date)})
 
        #print([func for func in dir(fc) if callable(getattr(fc, func))]) 
        
    elif diff < 14: # else parse bbc weather
        br = Browser()
        bbc_url = "https://www.bbc.com/weather/"
        br.open(bbc_url)
        br.set_handle_robots(False)
        br.addheaders = [("User-agent","Mozilla/5.0")] 
        
        formcount = 0
        for form in br.forms(): # get the number of form to input city name to
            if str(form.attrs["class"]) == "ls-o-form":
                break
            formcount += 1 
            
        br.select_form(nr=formcount-1)
        br.form["s"] = location     
        br.submit()
        url = br.geturl()

        resp = requests.get(url)
        bSoup = BeautifulSoup(resp.content, "html.parser")
        a = bSoup.select(".location-search-results__result__link")[0]
        final_url = bbc_url + a.attrs["href"]

        
        resp = requests.get(final_url)
        bSoup = BeautifulSoup(resp.content, "html.parser")
        temp_li = bSoup.select(f".wr-day--{diff}")[0] 
        
        weather_dict.update({"temp_celsius_max":int(temp_li.select(".wr-day-temperature__high")[0].select(".wr-value--temperature--c")[0].contents[0][:-1])})
        weather_dict.update({"temp_fahr_max":int(temp_li.select(".wr-day-temperature__high")[0].select(".wr-value--temperature--f")[0].contents[0][:-1])})
        weather_dict.update({"temp_celsius_min":int(temp_li.select(".wr-day-temperature__low-value")[0].select(".wr-value--temperature--c")[0].contents[0][:-1])})
        weather_dict.update({"temp_fahr_min":int(temp_li.select(".wr-day-temperature__low-value")[0].select(".wr-value--temperature--f")[0].contents[0][:-1])})
        
    return weather_dict
 
        
#getWeather("Kiev",datetime.datetime(2019, 7, 13, 12, 30)) use with time if less than five days 
#getWeather("Kiev,ua",datetime.datetime(2019, 7, 19)) without time if more

['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_will_be', 'get_forecast', 'get_weather_at', 'most_cold', 'most_hot', 'most_humid', 'most_rainy', 'most_snowy', 'most_windy', 'when_clear', 'when_clouds', 'when_ends', 'when_fog', 'when_hurricane', 'when_rain', 'when_snow', 'when_starts', 'when_storm', 'when_sun', 'when_tornado', 'will_be_clear_at', 'will_be_cloudy_at', 'will_be_foggy_at', 'will_be_hurricane_at', 'will_be_rainy_at', 'will_be_snowy_at', 'will_be_stormy_at', 'will_be_sunny_at', 'will_be_tornado_at', 'will_have_clear', 'will_have_clouds', 'will_have_fog', 'will_have_hurricane', 'will_have_rain', 'will_have_snow', 'will_have_storm', 'will_have_sun', 'will_have_tornado']


{'temp_celsius_prec': 18.25,
 'temp_fahr_prec': 64.85,
 'will_rain': False,
 'will_snow': False}

In [76]:
# def createData(input_data):
#     output_data = list()
#     for sent in input_data:
#         tokenized = nltk.word_tokenize(sent)
#         for token in tokenized:
#             inx = sent.index(token)
#             try:
#                 end = inx + sent[inx:].index(' ') - 1
#             except:
#                 end = -1
#             print(token, inx, end)
#         print("\n")
            